In [1]:
from PIL import Image;
import numpy as np;
import scipy as sci;
import csv;
import cv2;
import matplotlib.pyplot as plt;
import pandas as pd;
import glob

import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
inputDir = 'CandidateDataSet2/augmented_and_resized_RAW/2euro/'
input_file = glob.glob(inputDir+"*.jpg")
#input_file = [inputDir+"P70216-135401_Candidate_1.jpg"]  #For debugging

outputDir = 'CandidateDataSet2/augmented_and_resized_brightness_RAW/2euro/' #CHANGE IT!
brightness = [-100, -50, 0, 50, 100]
ROWS = 100
COLS = 100

for kfile in input_file:
    for kbright in brightness:
        tic()
        #print("Augmenting image "+str(kfile))
        im = Image.open(kfile) #Open image
        imarray = np.array(im)
        imarray = cv2.resize(imarray, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
        #imarray = cv2.erode(imarray, None, 1) 
        hsv = cv2.cvtColor(imarray, cv2.COLOR_BGR2HSV) #Convert it to hsv
        h, s, v = cv2.split(hsv)
        v = v.astype(np.int16)
        v += kbright
        v = np.maximum(np.zeros(v.shape),v)
        v = np.minimum(255*np.ones(v.shape),v)
        v = v.astype(np.uint8)
        final_hsv = cv2.merge((h, s, v))
        img_bright = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        #plt.imshow(img_bright)
        #plt.show()
        
        fileNamek = kfile.split("\\")[1]
        output_filename = outputDir+fileNamek.split(".")[0]+"_Brightness_"+str(kbright)+".jpg"
        img = Image.fromarray(img_bright, 'RGB')
        img.save(output_filename)
        #print(output_filename)
    #cv2.imwrite("image_processed.jpg", img)
    
    
    #for kAngle in angles:
    #    M = cv2.getRotationMatrix2D((imarray.shape[1]/2,imarray.shape[0]/2),kAngle,1)
    #    rotated_image = cv2.warpAffine(imarray,M,(imarray.shape[1],imarray.shape[0]))
    #    plt.imshow(rotated_image)
    #    plt.show()
    #    fileNamek = kfile.split("\\")[1]
    #    output_filename = outputDir+fileNamek.split(".")[0]+"_Aug_"+str(kAngle)+".jpg"
    #    img = Image.fromarray(rotated_image, 'RGB')
    #    img.save(output_filename)
print("Data augmentation with brightness completed!")    

Data augmentation with brightness completed!


In [ ]:
print(kfile)

# Labeling Data


In [ ]:
def readLabels(inputDir):
    #Reads the .csv files in order to initialize the data
    
    #For the train set
    fileName = inputDir + 'labels.csv'
    
    dfLabels = pd.read_csv(fileName)
    
    return dfLabels

dfLabels = readLabels(inputDir)
dfLabelsAug = pd.DataFrame();

for kName in range(0,len(dfLabels)):
    fileNameRoot = dfLabels.iloc[kName,0]
    fileNameRoot = fileNameRoot.split('.')[0]
    fileName0 = fileNameRoot + '_Aug_0.jpg'
    fileName90 = fileNameRoot + '_Aug_90.jpg'
    fileName180 = fileNameRoot + '_Aug_180.jpg'
    fileName270 = fileNameRoot + '_Aug_270.jpg'
    fileNameAug = [fileName0,fileName90,fileName180,fileName270]
    CoinorNoCoinAug = [dfLabels.iloc[kName,1],dfLabels.iloc[kName,1],dfLabels.iloc[kName,1],dfLabels.iloc[kName,1]]
    GroupAug = [dfLabels.iloc[kName,2],dfLabels.iloc[kName,2],dfLabels.iloc[kName,2],dfLabels.iloc[kName,2]]
    CoinTypeAug = [dfLabels.iloc[kName,3],dfLabels.iloc[kName,3],dfLabels.iloc[kName,3],dfLabels.iloc[kName,3]]
    dfLabelsAugk = pd.DataFrame();
    dfLabelsAugk['FileName']=fileNameAug
    dfLabelsAugk['CoinorNoCoin']=CoinorNoCoinAug
    dfLabelsAugk['Group']=GroupAug
    dfLabelsAugk['CoinType']=CoinTypeAug
    dfLabelsAug = dfLabelsAug.append(dfLabelsAugk)

In [ ]:
dfLabelsAug.to_csv(path_or_buf='labelsaug.csv',index=False)